# Instalación de paquetes de Python desde Jupyter notebook

En el actual mundo de la tecnología existe diversidad (dificilmente cuantificable) de ambientes de _software_ para cualquiera de los tantos lenguajes de pogramación existentes. Estos vienen con un _core_ de funcionalidades (librerías, paquetes, entre otras) para facilitar el proceso de desarrollo de código. Pero bien es sabido que, estas no siempre son las necesarias para la ejecución total de un desarrollo. Es por ello que se recurre a la instalación de librerías y paquetes y hacer utilizacón de las mismas. Pero algunas veces no suele ser tan sencillo darle ese plus al ambiente en el que se está trabajando.

Esto actualmente ocurre con _Jupyter Notebook_. Es muy recurrente encontrar en _StackOverflow_ cosas como: __"I installed package X and now I can't import it in the notebook. Help!"_ o _"how can I install a Python package so it works with my jupyter notebook, using pip and/or conda?"_

Fundamentalmente, la raíz del problema es debido a que el kernel de Jupyter está desconectado o se encuentra aislado de la _shell_ \; en otras palabras, el instalador apunta a una distinta versión de Python de la que está siendo utilizada por el _notebook_.

Ahora, se hará un inciso para incluir lo siguiente:
* _pip_
* _conda_

Ambos son manejadores de paquetes (no se entrará en detalle) pero cabe hacer la diferenciación:
* _pip_ instala paquetes de python en cualquier entorno.
* _conda_ instala cualquier paquete en entornos _conda_.

###### ¿Cuándo utilizar una u otra?
Si ya tienes una instalación de Python que estás utilizando, la elección será sencilla:
* Si la instalación de Python fue hecha utilizando Anaconda o Miniconda, entonces se usa _conda_ para instalar los paquetes de Python. Si _conda_ arroja que el "paquete no existe", se intenta con conda-forge.

* Si la instalación de Python fue de otra cualquier forma (desde fuente, pyenv, virtualenv, etc.), entonces se utiliza _pip_.

Cabe acotar que no es recomendable utilizar `sudo pip install`. __¡NUNCA!__

## Cómo usar conda desde Jupyter Notebook
Estando en el _notebook_ se puede necesitar la instalación de un paquete de Python con _conda_. El siguiente código muestra la notación para correr _conda_ directamente como comando de shell dentro del notebook:

In [ ]:
# DON'T DO THIS!
!conda install --yes numpy

Por varias razones descritas más adelante, esta forma de instalación no trabajará en la mayoría de los casos. Solo en los más simples.


En el siguiente se observará una forma en la que trabajará en general:

In [ ]:
# Install a conda package in the current Jupyter kernel
import sys
!conda install --yes --prefix {sys.prefix} numpy

¿Un poco más 'triquiñuela', no?
Pero ciertamente de esta forma instalará el paquete requerido en el actual _kernel_ de Jupyter que se encuentra en ejecución.

## Cómo usar pip desde Jupyter Notebook
Similarmente que en el caso con _conda_ , se puede estar tentado a utilizar:

In [ ]:
# DON'T DO THIS
!pip install numpy

Pero nuevamente, esto no trabajará en la mayoría de los casos. Solo en los más simples.


Ahora, su similar a hacerlo con _conda_ y que funcione en general:

In [ ]:
# Install a pip package in the current Jupyter kernel
import sys
!{sys.executable} -m pip install numpy

Y una vez más, con este paso se asegura que estás corriendo la versión de _pip_ asociada al _kernel_ de Python y los paquetes que han sido instalados funcionarán en _notebook_ actual.

# ¿Por qué la instalación de paquetes desde Jupyter es más tediosa?
Va a depender del sistema operativo, las instalaciones de Python y de Jupyter al ejecutarse. Pero en corto: esto es porque en Jupyter, el _shell environment_ se encuentra aislado del ejecutable de Python.

Comprendiendo unos pocos diferentes conceptos básicos para entrar más en detalle sobre lo complicado de hacer la instalación de paquetes desde Jupyter, debemos tener en cuenta lo siguiente:

1. Cómo el sistema operativo localiza los programas ejecutables.
2. Cómo Python instala y localiza paquetes.
3. Cómo Jupyter decide cuál Python ejecutable usar.


## Cómo el sistema operativo localiza los programas ejecutables
Cuando se utiliza la línea de comando y se tipea un comando como _python,_ _jupyter,_ _pip,_ _conda,_ etc., el sistema operativo contiene un "mecanismo" bien definido para encontrar el archivo ejecutable al cual refiere el nombre que se tipeó.

En sistemas Linux y MAC, el sistema primero chequea si hay coincidencias con un alias; si este falla al no encontrar, se referencia al entorno de variable $PATH.

In [6]:
!echo $PATH

/home/nelson/anaconda3/bin:/home/nelson/anaconda3/condabin:/home/nelson/.local/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/usr/games:/usr/local/games:/home/nelson/.dotnet/tools:/usr/lib/jvm/java-11-openjdk-amd64/bin


`$PATH` lista los directorios en orden, donde se buscará cualquier ejecutable. Por ejemplo, si te tipea `python`, con el `$PATH` de arriba se buscará `/home/nelson/anaconda3/bin/python`, y si no existe buscará `/home/nelson/anaconda3/condabin/python`, y así...


Si se quiere saber cual es el ejecutado al tipearse python, se puede utilizar el comando de consola:

In [9]:
!type python

python is /home/nelson/anaconda3/bin/python


## Cómo Python localiza los paquetes
Python utiliza un mecanismo similar para localizar los paquetes importados. La lista de paths buscados cuando se hace una importación, es encontrado:

In [11]:
import sys
sys.path

['/media/nelson/98B6A205B6A1E3CA/Users/Nelson/Documents/a_Semestre_II-2019/a_CIENCIA DE DATOS/Lab-II-Data-science',
 '/home/nelson/anaconda3/lib/python37.zip',
 '/home/nelson/anaconda3/lib/python3.7',
 '/home/nelson/anaconda3/lib/python3.7/lib-dynload',
 '',
 '/home/nelson/anaconda3/lib/python3.7/site-packages',
 '/home/nelson/anaconda3/lib/python3.7/site-packages/IPython/extensions',
 '/home/nelson/.ipython']

Por default, el primer lugar donde Python busca un módulo es en un path vacío, lo que significa el directorio actual. Y así va hasta encontrar el módulo buscado. Se puede encontrar cuál path ha sido utilizado con el atributo `path` de un módulo importado:

In [12]:
import numpy
numpy.__path__

['/home/nelson/anaconda3/lib/python3.7/site-packages/numpy']

En la mayoría de los casos, un paquete de Python que se instala con _pip_ o _conda_ se colocan en un directorio llamado `site-packages`. Hay que acotar que cada ejecutable de Python tiene su propio `site-packages`: lo que quiere decir que cuando se instala un _package,_ este se encuentra asociado con un particular ejecutable de Python y por lo tanto, será utilizado con esa instalación de Python.

Cuando se ejecuta `pip install` o `conda install` estos están asociados a una particular versión de Python:
* _pip_ instala paquetes en Python en su mismo _path_.
* _conda_ instala paquetes en el actual entorno de conda activo.

# Cómo Jupyter ejecuta código
Para hablar de cómo Jupyter selecciona el código de Python a ejecutar, se debe tener en cuenta el concepto de un _Jupyter kernel_.
Este es un conjunto de archivos que apuntan a Jupyter para ejecución de código dentro del _notebook._ Para Python kernels, este apuntará a una particular versión de Python.
Se puede cambiar el _kernel_ con: `Kernel → Choose Kernel menu item`.

Para visualizar los _kernels_ disponibles en el sitema:

In [13]:
!jupyter kernelspec list

Available kernels:
  python3    /home/nelson/anaconda3/share/jupyter/kernels/python3


# La causa del problema
¿Por qué `!pip install` o `!conda install` no siempre trabajan desde el _notebook?_

La respuesta es la siguiente: el _shell environment_ es determinado cuando el _Jupyter notebook_ es iniciado, mientras el _python executable_ es determinado por el _kernel,_ y los dos no necesariamente coinciden. En otras palabras, no hay garantías que `python`, `pip` y `conda` en tu `$PATH` serán compatibles con el ejecutable de Python usado por el _notebook_.

##### Teniendo en cuenta todo lo comentado anteriormente, se procederá con la instalación de las librerías solicitadas, acotando que serán instaladas con conda fuera de Jupyter, para luego hacer utilización de ellas.

Librerías a instalar:
1. Pandas
2. Seaborn

Se procede primero con la librería número uno.

In [ ]:
# Instalación
conda install pandas

#Importación
import pandas as pd

__Funciones de la librería pandas:__
* Panda is an open source library built on top of NumPy
* It allows for fast analysis and data cleaning and preparation
* It excels in performance and productivity
* It also has built-in visualization features
* It can work with data from a wide variety of sources

Ahora, se instalará la librería 2.

In [ ]:
# Instalación
conda install seaborn

# Importación
import seaborn as sb

__Funciones de la librería seaborn:__
* Seaborn is a statical plotting library
* It has beautiful default style
* It also is designed to work very well with pandas dataframe objects


PD: las funciones van de acuerdo a lo leído en su documentación oficial